In [11]:
import cv2
import datasets
import torch

import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

In [12]:
dataset = datasets.load_dataset("fuliucansheng/pascal_voc", "voc2007_main", split="train")
dataset = np.asarray(dataset)

/home/henning/.local/lib/python3.11/site-packages/datasets/load.py:1461: FutureWarning: The repository for fuliucansheng/pascal_voc contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/fuliucansheng/pascal_voc
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [30]:
def calc_IoU(A, B):
    def get_area(bbox):
        xl, yl, xu, yu = bbox
        return (xu - xl) * (yu - yl)
        
    xl_A, yl_A, xu_A, yu_A = A
    xl_B, yl_B, xu_B, yu_B = B

    #
    xl_A_hat = min(xl_A, xu_A)
    yl_A_hat = min(yl_A, yu_A)
    xu_A_hat = max(xu_A, xu_B)
    yu_A_hat = max(yu_A, yu_B)

    # Calculate area
    area_A = get_area(A)
    area_B = get_area(B)

    # Calculate intersection
    xli = max(xl_A_hat, xl_B)
    yli = max(yl_A_hat, yl_B)
    xui = min(xu_A_hat, xu_B)
    yui = min(yu_A_hat, yu_B)

    if xui > xli and yui > yli:
        intersection = (xui - xli) * (yui * yli)
    else:
        intersection = 0

    # Finding smallest enclosing bbox
    xl_C = min(xl_A_hat, xl_B)
    yl_C = min(yl_A_hat, yl_B)
    xu_C = max(xu_A_hat, xu_B)
    yu_C = max(yu_A_hat, yu_B)
    C = xl_C, yl_C, xu_C, yu_C

    area_C = get_area(C)

    union = area_A + area_B - intersection
    iou = intersection / union
    giou = iou - ((area_C - union) / area_C)

    return iou, giou

In [31]:
def show():
    for idx in range(100):
        element = dataset[idx]
        
        objects = element["objects"]
        bboxes = objects["bboxes"]
        bboxes_classes = objects["classes"]
    
        if len(bboxes_classes) < 2:
            continue
    
        img = element["image"]
        img = np.asarray(img, dtype=np.uint8)

        for xl, yl, xu, yu in bboxes:
            xl = int(xl)
            yl = int(yl)
            xu = int(xu)
            yu = int(yu)
            img = cv2.rectangle(img.copy(), (xl, yl), (xu, yu), (255, 0, 0), 1)
            
        coords = np.asarray(bboxes, dtype=int)
    
        fig, axes = plt.subplots(1, 1, figsize=(10, 10))
        axes.imshow(img)
        plt.show()
    
        break

In [35]:
ANCHOR = torch.tensor([-32, -32, 32, 32])
POS = torch.tensor([32, 32])
PRED = torch.randn(4, requires_grad=True)
GT = torch.tensor([0, 20, 100, 200])

ANCHOR[0] += POS[0]
ANCHOR[1] += POS[1]
ANCHOR[2] += POS[0]
ANCHOR[3] += POS[1]

PRED = ANCHOR + PRED

iou, giou = calc_IoU(PRED, GT)

L_iou = 1 - iou
L_giou = 1 - giou

print(f"{iou=}, {L_iou=}")
print(f"{giou=}, {L_giou=}")

iou=tensor(-1.0583, grad_fn=<DivBackward0>), L_iou=tensor(2.0583, grad_fn=<RsubBackward1>)
giou=tensor(-20.9522, grad_fn=<SubBackward0>), L_giou=tensor(21.9522, grad_fn=<RsubBackward1>)
